In [1]:
import mesh2sdf
import trimesh

filename = '../model/block.obj'
mesh = trimesh.load(filename, force='mesh')

In [10]:
mesh.vertices

TrackedArray([[-0.022632,  0.157543, -0.004347],
              [-0.075174,  0.167976, -0.025192],
              [-0.077239,  0.163912, -0.021716],
              ...,
              [-0.027723,  0.155172, -0.006445],
              [-0.072019,  0.150779, -0.042902],
              [-0.044036,  0.150708, -0.005709]])

In [12]:
for i,e in enumerate(mesh.vertices):
    print(i,e)

0 [-0.022632  0.157543 -0.004347]
1 [-0.075174  0.167976 -0.025192]
2 [-0.077239  0.163912 -0.021716]
3 [-0.016734  0.110903  0.040898]
4 [-0.076174  0.133648  0.052091]
5 [-0.067581  0.162514 -0.013916]
6 [-0.034495  0.127246  0.005205]
7 [-0.054989  0.044232  0.045269]
8 [-0.039189  0.043347  0.041535]
9 [0.006055 0.043707 0.045479]
10 [-0.039456  0.168482  0.002719]
11 [-0.015853  0.123285  0.031129]
12 [-0.022979  0.047265  0.051674]
13 [-0.010664  0.046795  0.047826]
14 [0.000109 0.044727 0.046148]
15 [-0.048237  0.118063  0.030649]
16 [-0.052978  0.047284  0.041192]
17 [-0.001039  0.048756  0.050981]
18 [-0.016677  0.049287  0.047116]
19 [-0.012694  0.049776  0.049879]
20 [-0.071538  0.119432  0.053754]
21 [-0.045683  0.048783  0.039154]
22 [-0.0425    0.050665  0.039371]
23 [-0.033737  0.050229  0.038469]
24 [-0.025961  0.048758  0.049144]
25 [-0.020594  0.050686  0.045394]
26 [-0.00118   0.052466  0.054806]
27 [0.041168 0.050011 0.032719]
28 [-0.038283  0.052792  0.03921 ]
29 [

In [9]:
mesh.faces

TrackedArray([[123, 691,  48],
              [704, 691, 123],
              [363,  82, 410],
              ...,
              [883, 891, 869],
              [960, 991, 959],
              [852, 858, 851]])